In [93]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [94]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

from enum import Enum
from pprint import pprint

from dotenv import load_dotenv


In [95]:
class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "중립"

In [96]:
enumParser = EnumOutputParser(enum=Sentiment)
format_instructions = enumParser.get_format_instructions()

In [97]:
template = """
영화 리뷰 텍스트를 입력받아 감정을 "긍정", "부정", "보통" 중 하나로 분류하는 시스템을 만드세요.

입력: "{input}"

출력 파서: {format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "중립" 중 하나의 단어만 출력하세요
2. 다른 설명이나 부가 설명을 추가하지 마세요
3. 이모지나 특수문자도 포함하지 마세요
4. 오직 하나의 단어만 출력하세요

"""

In [98]:
prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

In [99]:
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

In [100]:
fixing_parser = OutputFixingParser.from_llm(parser=enumParser, llm=model)

In [101]:
reviews = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
]

In [102]:
def safe_def(review, use_fixing_parser=True):
    try:
        chain = prompt | model | (fixing_parser if use_fixing_parser else enumParser)
        result = chain.invoke({"input": review})
        
        return result, None
    
    except OutputParserException as e:
        return None, f"파싱 오류: {e}"
    except Exception as e:
        return None, f"일반 오류: {e}"


In [103]:
def run_def():
    
    success = 0
    total = len(reviews)
    
    positive=0
    negative=0
    neutral=0
    
    for i, review in enumerate(reviews,1):
        print(f"\n{i}.  {review}")
        
        result, error = safe_def(review, use_fixing_parser=True)
        
        if result:
            print(f"    감정: {result.value}")
            success+=1
            
            if result == "긍정":
                positive += 1
            elif result == "부정":
                negative += 1
            else:
                neutral += 1
        else: 
            print(f"    오류: {error}")
            
            re_result, re_error = safe_def(review, use_fixing_parser=False)
            
            if re_result:
                print(f"    감정: {re_result}")
                success += 1
                if re_result == "긍정":
                    positive += 1
                elif re_result == "부정":
                    negative += 1
                else:
                    neutral += 1
            else:
                print(f"    재시도 실패: {re_error}")

    print("결과")
    print(f"긍정: {positive}")
    print(f"부정: {negative}")
    print(f"중립: {neutral}")
    print(f"성공: {success}/{total} ({success/total*100:.1f}%)")
    print(f"실패: {total-success}/{total} ({(total - success) / total*100:.1f}%)")
    

In [104]:
try: 
    run_def()
except Exception as e:
    print(f"오류 내용: {e}")


1.  이 영화 정말 재미없어요. 시간 낭비였습니다.
    감정: 부정

2.  배우들의 연기가 훌륭하고 스토리도 감동적이었어요!
    감정: 긍정

3.  그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.
    감정: 중립
결과
긍정: 1
부정: 1
중립: 1
성공: 3/3 (100.0%)
실패: 0/3 (0.0%)
